In [1]:
import os
import json
import pandas as pd
import torch
from torch_geometric.data import Dataset, download_url
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import convert
import networkx as nx 
import itertools
import math

from torch_geometric.data import InMemoryDataset
from ast import literal_eval
from sentence_transformers import SentenceTransformer

In [ ]:
# Sources
# https://medium.com/cj-express-tech-tildi/first-timers-guide-to-pytorch-geometric-part-1-the-basic-1b6006e1f4db

# Ground truth UB normal:
# 1 is normal
# 0 is abnormal

In [33]:
def get_jsons(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations.
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist


def normalize_keypoints(keypoints):
    # Transform 1: Set Nose as origin
    nose_index = order.index('Nose')
    nose_co, _ = keypoints[nose_index]
    tmp_keypoints = []
    for kp in keypoints:
        co, confidence = kp
        adj_x = co[0] - nose_co[0]
        adj_y = co[1] - nose_co[1]
        adj_co = (adj_x, adj_y)
        tmp_keypoints.append((adj_co, confidence))
    # Transofmr 2: Standardize pose size.
    # This is done by setting the distance between the shoulders to 1
    rshoulder_index = order.index('RShoulder')
    lshoulder_index = order.index('LShoulder')
    co1 = keypoints[rshoulder_index][0]
    co2 = keypoints[lshoulder_index][0]
    distance = math.dist(co1, co2)
    normalized_keypoints = []
    for kp in tmp_keypoints:
        co, confidence = kp
        adj_x = co[0]/distance
        adj_y = co[1]/distance
        adj_co = (adj_x, adj_y)
        normalized_keypoints.append((adj_co, confidence))
    return normalized_keypoints


def reformat_kp(key_points):
    """Reformat the keypoints so that the array
    contains 1 item per keypoints.
    Each item should have the following format containing
    the coordinates the first element and the confidence as the
    second element:
    Tuple[Tuple[float, float], float]
    """
    co_keypoints = []
    for i in range(0, len(key_points), 3):
        x = float(key_points[i])
        y = float(key_points[i+1])
        c = float(key_points[i+2])
        co = (x,y)
        co_keypoints.append((co, c))
    return co_keypoints

def load_tracking_jsons(json_files):
    """
    Load in the jsons with the corresponding ground truth
    """
    dfs = []
    for file in json_files:
        if file.endswith('tracked_person.json'):
            video = '_'.join(file.split('/')[-1].split('-')[0].split('_')[0:-1])
            with open(file) as f:
                tracking = json.load(f)
             # The ground truth shared in the repo of STG-NF contains two formats
            # One is a Numpy array, the other is a text file containing per person if which frame contains invalid poses
            # As we want to focus on detecting abnormallies in the pose graph,
            # we only want to have the videos with a grond truth on person basis.
            gt_file = f'./data/UBnormal/annotations/{video.replace("alphapose_tracked", "tracks")}.txt'
            try:
                np.load(gt_file)
                continue
            # If there is no ground truth, ignore the video
            except FileNotFoundError:
                continue
            except ValueError:
                with open(gt_file) as file:
                    lines = [line.strip() for line in file]
                gt_video = {}
                for gt in lines:
                    person, start_frame, end_frame = gt.split(',')
                    # Use float to deal with scientific notation
                    person = int(float(person))
                    start_frame = int(float(start_frame))
                    end_frame = int(float(end_frame))
                    gt_video[person] = (start_frame, end_frame)
            items = []
            for person, tracking_data in tracking.items():
                # If it's not in the gt dict, it means that the person has a normal pose
                # for the whole video aka there is no frame in which the person has an abnormal frame
                start_frame, end_frame = gt_video.get(int(person), (-1, -1)) 
                for frame, data in tracking_data.items():
                    frame_number = int(frame)
                    data['frame'] = frame_number
                    if start_frame <= frame_number <= end_frame:
                        data['label'] = 'abnormal'
                    else:
                        data['label'] = 'normal'
                    items.append(data)
            if items:
                df = pd.DataFrame(items)
                df['video'] = video
                df = df[['video', 'frame', 'label', 'keypoints', 'scores']]
                dfs.append(df)
    df_overview = pd.concat(dfs, ignore_index=True)
    return df_overview


def get_edge_weight(graph):
    edges = nx.get_edge_attributes(graph, 'weight')
    edge_weights = []
    # Ensure order of list is the same
    for source, target in connections:
        n1 = order[source]
        n2 = order[target]
        try:
            weight = edges[(n1, n2)]
        # as the direction doesn't matter in this graph, it's possible that the keys are stored in a different order 
        # in the graph
        except KeyError:
            weight = edges[(n2, n1)]
        edge_weights.append(weight)
    return edge_weights

In [11]:
df_provided_poses = load_tracking_jsons(get_jsons('./data/UBnormal/poses'))

In [5]:
df_training = df_provided_poses.sample(frac=0.80)
df_subset =  df_provided_poses.drop(df_training.index)
df_validation = df_subset.sample(frac=0.5)
df_test = df_subset.drop(df_validation.index)

In [6]:
df_training.to_csv('./PyGod/data/UBnormal/training.csv', index=False)
df_validation.to_csv('./PyGod/data/UBnormal/validation.csv', index=False)
df_test.to_csv('./PyGod/data/UBnormal/testing.csv', index=False)

In [7]:
df_test.shape

(22929, 5)

In [21]:
# Labels of the keypoints in order
# Source https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
raw_order = [
    {0,  "Nose"},
    {1,  "LEye"},
    {2,  "REye"},
    {3,  "LEar"},
    {4,  "REar"},
    {5,  "LShoulder"},
    {6,  "RShoulder"},
    {7,  "LElbow"},
    {8,  "RElbow"},
    {9,  "LWrist"},
    {10, "RWrist"},
    {11, "LHip"},
    {12, "RHip"},
    {13, "LKnee"},
    {14, "Rknee"},
    {15, "LAnkle"},
    {16, "RAnkle"},
]
order = []
for s in raw_order:
    for item in s:
        if isinstance(item, str):
            order.append(item)
            break

connections = [list(perm) for perm in itertools.permutations([i for i in range(0, 17)], 2)]

class NameEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, name_array):
        x = self.model.encode(name_array, show_progress_bar=False,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()


def convert_keypoints_to_graph(key_points):
    """Transforms the keypoints in a more usable format
    """
    graph = nx.Graph()
    for i, kp in enumerate(key_points):
        pos = kp[0]
        name = order[i]
        graph.add_node(node_for_adding=name,
                       pos=(pos[0], -pos[1]))
    for source, target in connections:
        n1 = order[source]
        n2 = order[target]
        co1 = key_points[source][0]
        co2 = key_points[target][0]
        weight = math.dist(co1, co2)
        graph.add_edge(n1, n2, weight=weight)
    return graph
        
class PoseDataset(InMemoryDataset):
    def __init__(self, raw_file_name, root, transform=None, pre_transform=None, pre_filter=None):
        # the root argument should point to the directory where you have saved the data or want to save it
        self.raw_file_name = raw_file_name
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [self.raw_file_name]

    @property
    def processed_file_names(self):
        target_file = self.raw_file_names[0]
        file_name = target_file.split('/')[-1].split('.')[0]
        filelist = [];
        for root, dirs, files in os.walk(self.root):
            for file in files:
                if file_name in file and file.endswith('pt'):
                    # json_file = os.path.join(root,file)
                    filelist.append(file)
        return filelist

    @property
    def num_classes(self):
        return 2
    
    def process(self):
        # Read data into huge `Data` list.
        name_encoder = NameEncoder()
        for file_number, file in enumerate(self.raw_file_names):
            file_name = file.split('/')[-1].split('.')[0]
            df_data = pd.read_csv(file, chunksize=1000)
            for chunk_index, df_chunk in enumerate(df_data):
                data_list = []
                for row_index, row in df_chunk.iterrows():
                    kp = row.normalized_keypoints
                    if isinstance(row.normalized_keypoints, str):
                        kp = literal_eval(row.normalized_keypoints)
                    graph = convert_keypoints_to_graph(kp)
                    data = convert.from_networkx(graph)
                    data.y = 1 if row.label == 'normal' else 0,
                    xs = [name_encoder([node for node in graph.nodes()])]
                    data.x = torch.cat(xs, dim=-1)
                    # data = Data(
                    #     x=x,
                    #    edge_index=torch.from_numpy(np.array(connections)),
                    #      edge_attr=torch.from_numpy(np.array([[item] for item in get_edge_weight(graph)])),
                    #     y=1 if row.label == 'normal' else 0,
                    #     pos=torch.from_numpy(positions ),
                    # )
                    data_list.append(data)
                
                if self.pre_filter is not None:
                    data_list = [data for data in data_list if self.pre_filter(data)]
                
                if self.pre_transform is not None:
                    data_list = [self.pre_transform(data) for data in data_list]
                self.save(data_list, f'./PyGod/data/UBnormal/processed/{file_name}_{chunk_index}.pt')
                print(f"Saved data to ./PyGod/data/UBnormal/processed/{file_name}_{chunk_index}.pt")


In [125]:
test_dataset = PoseDataset('./PyGod/data/UBnormal/testing.csv', './PyGod/data/UBnormal')

Processing...
/home/ruben/miniconda3/envs/pose_graph_anomaly/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved data to ./PyGod/data/UBnormal/processed/testing_0.pt


Done!


In [127]:
validation_dataset = PoseDataset('./PyGod/data/UBnormal/validation.csv', './PyGod/data/UBnormal')

Processing...
/home/ruben/miniconda3/envs/pose_graph_anomaly/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved data to ./PyGod/data/UBnormal/processed/validation_0.pt


Done!


In [22]:
training_dataset = PoseDataset('./PyGod/data/UBnormal/training.csv', './PyGod/data/UBnormal')

Processing...
/home/ruben/miniconda3/envs/pose_graph_anomaly/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saved data to ./PyGod/data/UBnormal/processed/training_0.pt
Saved data to ./PyGod/data/UBnormal/processed/training_1.pt
Saved data to ./PyGod/data/UBnormal/processed/training_2.pt
Saved data to ./PyGod/data/UBnormal/processed/training_3.pt
Saved data to ./PyGod/data/UBnormal/processed/training_4.pt
Saved data to ./PyGod/data/UBnormal/processed/training_5.pt
Saved data to ./PyGod/data/UBnormal/processed/training_6.pt
Saved data to ./PyGod/data/UBnormal/processed/training_7.pt
Saved data to ./PyGod/data/UBnormal/processed/training_8.pt
Saved data to ./PyGod/data/UBnormal/processed/training_9.pt
Saved data to ./PyGod/data/UBnormal/processed/training_10.pt
Saved data to ./PyGod/data/UBnormal/processed/training_11.pt
Saved data to ./PyGod/data/UBnormal/processed/training_12.pt
Saved data to ./PyGod/data/UBnormal/processed/training_13.pt
Saved data to ./PyGod/data/UBnormal/processed/training_14.pt
Saved data to ./PyGod/data/UBnormal/processed/training_15.pt
Saved data to ./PyGod/data/UBnorma

Done!


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [62]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(training_dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, training_dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
# To TEst:
# DONE, AdONE, CoLA, CONAD

In [23]:
training_dataset.num_classes

2

In [7]:
from pygod.detector import DONE

In [8]:
detector = DONE(device_map="auto", torch_dtype=torch.float16)

In [27]:
detector.fit(training_dataset[0])

ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

In [28]:
!pip install torch-sparse

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [79 lines of output]
      running bdist_wheel
      /home/ruben/miniconda3/envs/pose_graph_anomaly/lib/python3.8/site-packages/torch/utils/cpp_extension.py:499: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.8
      creating build/lib.linux-x86_64-3.8/torch_sparse
      copying torch_sparse/mul.py -> build/lib.linux-x86_64-3.8/torch_sparse
      copying torch_sparse/spmm.py -> build/lib.linux-x86_64-3.8/torch_sparse
      copying torch_sparse/masked_select.py -> build/lib.linu

In [139]:
detector = DOMINANT(hid_dim=64, num_layers=4, epoch=100)

In [26]:
detector.fit(training_dataset)

ValueError: Expected a 'Data', 'HeteroData', or a tuple of 'FeatureStore' and 'GraphStore' (got '<class '__main__.PoseDataset'>')

In [137]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = training_dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

AttributeError: 'GlobalStorage' object has no attribute 'train_mask'

In [121]:
test_dataset[0]

Data(edge_index=[2, 272], pos=[17, 2], weight=[272], y=[1])